In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pulp
import sys
import os
import importlib.util
import importlib
import gurobipy as gp

# モジュールを強制的に再ロード（古いキャッシュをクリア）
if 'code' in sys.modules:
    del sys.modules['code']
# サブモジュールも削除
for module_name in list(sys.modules.keys()):
    if module_name.startswith('code.'):
        del sys.modules[module_name]

current_dir = os.getcwd()
spec = importlib.util.spec_from_file_location("code", os.path.join(current_dir, "code", "__init__.py"))
code_module = importlib.util.module_from_spec(spec)
sys.modules["code"] = code_module
spec.loader.exec_module(code_module)

# サブモジュールも再ロード
importlib.reload(code_module)

make_tensor_grid_7d = code_module.make_tensor_grid_7d
product_beta_density = code_module.product_beta_density
solve_mechanism_3goods_4synergy = code_module.solve_mechanism_3goods_4synergy
solve_mechanism_3goods_4synergy_iterative = code_module.solve_mechanism_3goods_4synergy_iterative
plot_polyhedral_regions = code_module.plot_polyhedral_regions

print("モジュールを再ロードしました")

モジュールを再ロードしました


In [2]:
home_license = os.path.expanduser('~/gurobi.lic')
os.environ['GRB_LICENSE_FILE'] = home_license

SOLVER = pulp.GUROBI(msg=True)


In [3]:
# ケース: 3財4シナジー
# 各次元のグリッドサイズ（7次元すべて同じサイズ）
N = 3

# 各次元のBeta分布パラメータ
# [財a, 財b, 財c, シナジーα12, シナジーα23, シナジーα31, シナジーα123]
BETA_PARAMS = [
    (1.0, 1.0),  # 財a
    (1.0, 1.0),  # 財b
    (1.0, 1.0),  # 財c
    (1.0, 1.0),  # シナジーα12
    (1.0, 1.0),  # シナジーα23
    (1.0, 1.0),  # シナジーα31
    (1.0, 1.0),  # シナジーα123
]

In [4]:
# グリッド点と重みを生成
points, weights = make_tensor_grid_7d(N, BETA_PARAMS)
print(f"#types = {len(points)}")
print("ex. first point, weight =", points[0], weights[0])

# 反復的制約追加アルゴリズムで最適オークション機構を求解
status, obj_val, u_sol, p_sol, n_iter = solve_mechanism_3goods_4synergy_iterative(
    points, weights, grid_size=N, solver=SOLVER
)

print("\nLP status:", status)
print("Optimal revenue:", obj_val)
print(f"Number of iterations: {n_iter}")

#types = 2187
ex. first point, weight = (0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666) 0.0004572473708276177
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2749041
Academic license 2749041 - for non-commercial use only - registered to ko___@u.northwestern.edu
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[arm] - Darwin 25.1.0 25B78)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2749041 - for non-commercial use only - registered to ko___@u.northwestern.edu
Optimize a model with 55404 rows, 17496 columns and 155277 nonzeros (Max)
Model fingerprint: 0x19e266a2
Model has 17496 linear objective coefficients
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [8e-05, 5e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 